In [0]:
# MAGIC %md
# MAGIC # 05 - Model Monitoring & Performance Tracking
# MAGIC 
# MAGIC **Monitor production model performance and detect drift**
# MAGIC 
# MAGIC ## Objectives:
# MAGIC - Track prediction distributions over time
# MAGIC - Monitor data quality and feature drift
# MAGIC - Detect model performance degradation
# MAGIC - Generate monitoring reports
# MAGIC - Setup alerting thresholds


In [0]:
%restart_python

In [0]:

# Standard imports
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports complete")


In [0]:

# MAGIC %md
# MAGIC ## 2. Project Setup


In [0]:

print("="*60)
print("PROJECT SETUP")
print("="*60)

# Define project root
project_root = "/Workspace/COMM - Commercial Analytics (CMAN)/MMM Quattro 2025/Satish/MLFLOW_sample"

# Add to path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"\n📂 Project root: {project_root}")
print(f"✅ Added to sys.path")

# Import custom modules
from src.utils import ConfigLoader, DataLoader, MLflowLogger, safe_display

print(f"✅ Custom modules imported")
print("="*60)

In [0]:

# MAGIC %md
# MAGIC ## 3. Load Configuration


In [0]:

print("="*60)
print("LOADING CONFIGURATION")
print("="*60)

config_path = f'{project_root}/config/config.yaml'
config = ConfigLoader.load_config(config_path)

print(f"\n✅ Configuration loaded")
print(f"  • Project: {config['project']['name']}")
print(f"  • Model Registry: {config['mlflow']['model_registry_name']}")
print("="*60)


In [0]:
# MAGIC %md
# MAGIC ## 4. Setup MLflow


In [0]:

print("="*60)
print("SETTING UP MLFLOW")
print("="*60)

experiment_name = config['mlflow']['experiment_name']
mlflow.set_experiment(experiment_name)

# Initialize MLflow client
client = MlflowClient()

print(f"\n✅ MLflow experiment set: {experiment_name}")

experiment = mlflow.get_experiment_by_name(experiment_name)
print(f"  • Experiment ID: {experiment.experiment_id}")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 5. Load Production Model & Data



In [0]:
print("="*60)
print("LOADING PRODUCTION MODEL & DATA")
print("="*60)

model_name = config['mlflow']['model_registry_name']
processed_path = config['data']['processed_path']

# Load production model
model_uri = f"models:/{model_name}/Production"
production_model = mlflow.sklearn.load_model(model_uri)

print(f"\n✅ Production model loaded")
print(f"  • Model: {model_name}")
print(f"  • Type: {type(production_model).__name__}")

# Load training data (baseline)
X_train = pd.read_csv(f"{processed_path}X_train.csv")
y_train = pd.read_csv(f"{processed_path}y_train.csv").squeeze()

# Load test data (current/production)
X_test = pd.read_csv(f"{processed_path}X_test.csv")
y_test = pd.read_csv(f"{processed_path}y_test.csv").squeeze()

print(f"\n✅ Data loaded")
print(f"  • Training set: {X_train.shape}")
print(f"  • Test set: {X_test.shape}")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 6. Baseline Performance Metrics


In [0]:

print("="*60)
print("CALCULATING BASELINE METRICS")
print("="*60)

# Get baseline predictions (training)
y_train_pred = production_model.predict(X_train)

# Calculate baseline metrics
baseline_metrics = {
    'rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
    'mae': mean_absolute_error(y_train, y_train_pred),
    'r2': r2_score(y_train, y_train_pred),
    'mean_prediction': y_train_pred.mean(),
    'std_prediction': y_train_pred.std(),
    'mean_actual': y_train.mean(),
    'std_actual': y_train.std()
}

print(f"\n📊 Baseline Metrics (Training Data):")
print(f"  • RMSE:              ${baseline_metrics['rmse']:,.2f}")
print(f"  • MAE:               ${baseline_metrics['mae']:,.2f}")
print(f"  • R²:                {baseline_metrics['r2']:.4f}")
print(f"  • Mean Prediction:   ${baseline_metrics['mean_prediction']:,.2f}")
print(f"  • Std Prediction:    ${baseline_metrics['std_prediction']:,.2f}")
print(f"  • Mean Actual:       ${baseline_metrics['mean_actual']:,.2f}")
print(f"  • Std Actual:        ${baseline_metrics['std_actual']:,.2f}")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 7. Current Performance Metrics


In [0]:

print("="*60)
print("CALCULATING CURRENT METRICS")
print("="*60)

# Get current predictions (test/production)
y_test_pred = production_model.predict(X_test)

# Calculate current metrics
current_metrics = {
    'rmse': np.sqrt(mean_squared_error(y_test, y_test_pred)),
    'mae': mean_absolute_error(y_test, y_test_pred),
    'r2': r2_score(y_test, y_test_pred),
    'mean_prediction': y_test_pred.mean(),
    'std_prediction': y_test_pred.std(),
    'mean_actual': y_test.mean(),
    'std_actual': y_test.std()
}

print(f"\n📊 Current Metrics (Test/Production Data):")
print(f"  • RMSE:              ${current_metrics['rmse']:,.2f}")
print(f"  • MAE:               ${current_metrics['mae']:,.2f}")
print(f"  • R²:                {current_metrics['r2']:.4f}")
print(f"  • Mean Prediction:   ${current_metrics['mean_prediction']:,.2f}")
print(f"  • Std Prediction:    ${current_metrics['std_prediction']:,.2f}")
print(f"  • Mean Actual:       ${current_metrics['mean_actual']:,.2f}")
print(f"  • Std Actual:        ${current_metrics['std_actual']:,.2f}")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 8. Performance Comparison & Drift Detection


In [0]:

print("="*60)
print("PERFORMANCE DRIFT ANALYSIS")
print("="*60)

# Calculate percentage changes
drift_metrics = {
    'rmse_change_pct': ((current_metrics['rmse'] - baseline_metrics['rmse']) / baseline_metrics['rmse']) * 100,
    'mae_change_pct': ((current_metrics['mae'] - baseline_metrics['mae']) / baseline_metrics['mae']) * 100,
    'r2_change_pct': ((current_metrics['r2'] - baseline_metrics['r2']) / baseline_metrics['r2']) * 100,
    'mean_pred_change_pct': ((current_metrics['mean_prediction'] - baseline_metrics['mean_prediction']) / baseline_metrics['mean_prediction']) * 100,
    'std_pred_change_pct': ((current_metrics['std_prediction'] - baseline_metrics['std_prediction']) / baseline_metrics['std_prediction']) * 100
}

print(f"\n📈 Performance Changes (Baseline → Current):")
print(f"  • RMSE Change:       {drift_metrics['rmse_change_pct']:+.2f}%")
print(f"  • MAE Change:        {drift_metrics['mae_change_pct']:+.2f}%")
print(f"  • R² Change:         {drift_metrics['r2_change_pct']:+.2f}%")
print(f"  • Mean Pred Change:  {drift_metrics['mean_pred_change_pct']:+.2f}%")
print(f"  • Std Pred Change:   {drift_metrics['std_pred_change_pct']:+.2f}%")

# Define alert thresholds
ALERT_THRESHOLDS = {
    'rmse_increase': 10,  # Alert if RMSE increases by >10%
    'mae_increase': 10,   # Alert if MAE increases by >10%
    'r2_decrease': 5,     # Alert if R² decreases by >5%
    'prediction_shift': 15  # Alert if mean prediction shifts by >15%
}

# Check for alerts
alerts = []

if drift_metrics['rmse_change_pct'] > ALERT_THRESHOLDS['rmse_increase']:
    alerts.append(f"⚠️ RMSE increased by {drift_metrics['rmse_change_pct']:.2f}% (threshold: {ALERT_THRESHOLDS['rmse_increase']}%)")

if drift_metrics['mae_change_pct'] > ALERT_THRESHOLDS['mae_increase']:
    alerts.append(f"⚠️ MAE increased by {drift_metrics['mae_change_pct']:.2f}% (threshold: {ALERT_THRESHOLDS['mae_increase']}%)")

if drift_metrics['r2_change_pct'] < -ALERT_THRESHOLDS['r2_decrease']:
    alerts.append(f"⚠️ R² decreased by {abs(drift_metrics['r2_change_pct']):.2f}% (threshold: {ALERT_THRESHOLDS['r2_decrease']}%)")

if abs(drift_metrics['mean_pred_change_pct']) > ALERT_THRESHOLDS['prediction_shift']:
    alerts.append(f"⚠️ Mean prediction shifted by {drift_metrics['mean_pred_change_pct']:+.2f}% (threshold: {ALERT_THRESHOLDS['prediction_shift']}%)")

if alerts:
    print(f"\n🚨 ALERTS DETECTED:")
    for alert in alerts:
        print(f"  {alert}")
else:
    print(f"\n✅ No alerts - Model performance is stable")

print("="*60)

In [0]:

# MAGIC %md
# MAGIC ## 9. Feature Drift Analysis


In [0]:

print("="*60)
print("FEATURE DRIFT ANALYSIS")
print("="*60)

# Calculate feature statistics
feature_drift = []

for col in X_train.columns:
    # Baseline (training) statistics
    baseline_mean = X_train[col].mean()
    baseline_std = X_train[col].std()
    
    # Current (test) statistics
    current_mean = X_test[col].mean()
    current_std = X_test[col].std()
    
    # Calculate changes
    mean_change_pct = ((current_mean - baseline_mean) / baseline_mean * 100) if baseline_mean != 0 else 0
    std_change_pct = ((current_std - baseline_std) / baseline_std * 100) if baseline_std != 0 else 0
    
    # Kolmogorov-Smirnov test for distribution drift
    ks_statistic, ks_pvalue = stats.ks_2samp(X_train[col], X_test[col])
    
    feature_drift.append({
        'Feature': col,
        'Baseline_Mean': baseline_mean,
        'Current_Mean': current_mean,
        'Mean_Change_%': mean_change_pct,
        'Baseline_Std': baseline_std,
        'Current_Std': current_std,
        'Std_Change_%': std_change_pct,
        'KS_Statistic': ks_statistic,
        'KS_PValue': ks_pvalue,
        'Drift_Detected': 'Yes' if ks_pvalue < 0.05 else 'No'
    })

feature_drift_df = pd.DataFrame(feature_drift)

print(f"\n📊 Feature Drift Summary:")
safe_display(feature_drift_df)

# Identify features with significant drift
drifted_features = feature_drift_df[feature_drift_df['Drift_Detected'] == 'Yes']

if len(drifted_features) > 0:
    print(f"\n⚠️ Features with Significant Drift ({len(drifted_features)}):")
    safe_display(drifted_features[['Feature', 'Mean_Change_%', 'KS_Statistic', 'KS_PValue']])
else:
    print(f"\n✅ No significant feature drift detected")

print("="*60)

In [0]:

# MAGIC %md
# MAGIC ## 10. Prediction Distribution Analysis


In [0]:

print("="*60)
print("PREDICTION DISTRIBUTION ANALYSIS")
print("="*60)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Prediction distributions comparison
axes[0, 0].hist(y_train_pred, bins=30, alpha=0.6, label='Baseline (Training)', edgecolor='black')
axes[0, 0].hist(y_test_pred, bins=30, alpha=0.6, label='Current (Test)', edgecolor='black')
axes[0, 0].set_xlabel('Predicted Price ($)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Prediction Distribution Comparison', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Actual vs Predicted - Baseline
axes[0, 1].scatter(y_train, y_train_pred, alpha=0.5, s=30, label='Baseline')
min_val = min(y_train.min(), y_train_pred.min())
max_val = max(y_train.max(), y_train_pred.max())
axes[0, 1].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
axes[0, 1].set_xlabel('Actual Price ($)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Predicted Price ($)', fontsize=11, fontweight='bold')
axes[0, 1].set_title(f'Baseline: R² = {baseline_metrics["r2"]:.4f}', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Actual vs Predicted - Current
axes[1, 0].scatter(y_test, y_test_pred, alpha=0.5, s=30, color='orange', label='Current')
min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Price ($)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Predicted Price ($)', fontsize=11, fontweight='bold')
axes[1, 0].set_title(f'Current: R² = {current_metrics["r2"]:.4f}', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Error distribution comparison
baseline_errors = y_train - y_train_pred
current_errors = y_test - y_test_pred

axes[1, 1].hist(baseline_errors, bins=30, alpha=0.6, label='Baseline Errors', edgecolor='black')
axes[1, 1].hist(current_errors, bins=30, alpha=0.6, label='Current Errors', edgecolor='black')
axes[1, 1].axvline(x=0, color='r', linestyle='--', lw=2)
axes[1, 1].set_xlabel('Prediction Error ($)', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Error Distribution Comparison', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("✅ Prediction distribution analysis complete")
print("="*60)

In [0]:

# MAGIC %md
# MAGIC ## 11. Data Quality Monitoring


In [0]:

print("="*60)
print("DATA QUALITY MONITORING")
print("="*60)

# Check for data quality issues
quality_issues = []

# 1. Check for missing values
missing_baseline = X_train.isnull().sum().sum()
missing_current = X_test.isnull().sum().sum()

print(f"\n📊 Missing Values:")
print(f"  • Baseline: {missing_baseline}")
print(f"  • Current:  {missing_current}")

if missing_current > 0:
    quality_issues.append(f"⚠️ Current data has {missing_current} missing values")

# 2. Check for outliers (using IQR method)
outlier_counts = {}

for col in X_train.columns:
    Q1 = X_train[col].quantile(0.25)
    Q3 = X_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_current = ((X_test[col] < lower_bound) | (X_test[col] > upper_bound)).sum()
    outlier_counts[col] = outliers_current

outlier_df = pd.DataFrame({
    'Feature': list(outlier_counts.keys()),
    'Outlier_Count': list(outlier_counts.values()),
    'Outlier_Pct': [v/len(X_test)*100 for v in outlier_counts.values()]
}).sort_values('Outlier_Count', ascending=False)

print(f"\n📊 Outliers in Current Data:")
safe_display(outlier_df)

# Check for excessive outliers
excessive_outliers = outlier_df[outlier_df['Outlier_Pct'] > 10]
if len(excessive_outliers) > 0:
    quality_issues.append(f"⚠️ {len(excessive_outliers)} features have >10% outliers")

# 3. Check for data range violations
range_violations = []

for col in X_train.columns:
    baseline_min = X_train[col].min()
    baseline_max = X_train[col].max()
    current_min = X_test[col].min()
    current_max = X_test[col].max()
    
    if current_min < baseline_min or current_max > baseline_max:
        range_violations.append({
            'Feature': col,
            'Baseline_Range': f"[{baseline_min:.2f}, {baseline_max:.2f}]",
            'Current_Range': f"[{current_min:.2f}, {current_max:.2f}]",
            'Out_of_Range': 'Yes'
        })

if range_violations:
    range_violations_df = pd.DataFrame(range_violations)
    print(f"\n⚠️ Features with Out-of-Range Values:")
    safe_display(range_violations_df)
    quality_issues.append(f"⚠️ {len(range_violations)} features have out-of-range values")
else:
    print(f"\n✅ All features within expected ranges")

# Summary
if quality_issues:
    print(f"\n🚨 DATA QUALITY ISSUES DETECTED:")
    for issue in quality_issues:
        print(f"  {issue}")
else:
    print(f"\n✅ No data quality issues detected")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 12. Generate Monitoring Report


In [0]:

print("="*60)
print("GENERATING MONITORING REPORT")
print("="*60)

# Create comprehensive monitoring report
monitoring_report = {
    'report_timestamp': datetime.now().isoformat(),
    'model_name': model_name,
    'model_type': type(production_model).__name__,
    
    'baseline_metrics': baseline_metrics,
    'current_metrics': current_metrics,
    'drift_metrics': drift_metrics,
    
    'alerts': alerts,
    'alert_count': len(alerts),
    
    'feature_drift_summary': {
        'total_features': len(feature_drift_df),
        'drifted_features': len(drifted_features),
        'drift_percentage': (len(drifted_features) / len(feature_drift_df) * 100) if len(feature_drift_df) > 0 else 0
    },
    
    'data_quality': {
        'missing_values': int(missing_current),
        'quality_issues': quality_issues,
        'issue_count': len(quality_issues)
    },
    
    'recommendation': 'Model retraining recommended' if (len(alerts) > 0 or len(drifted_features) > 2) else 'Model performance is stable'
}

# Save report as JSON
import json
report_path = f"{processed_path}monitoring_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(report_path, 'w') as f:
    json.dump(monitoring_report, f, indent=2, default=str)

print(f"\n✅ Monitoring report saved: {report_path}")

# Save feature drift details
drift_path = f"{processed_path}feature_drift_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
feature_drift_df.to_csv(drift_path, index=False)
print(f"✅ Feature drift details saved: {drift_path}")

# Log to MLflow
with mlflow.start_run(run_name="model_monitoring"):
    # Log metrics
    MLflowLogger.log_metrics_from_dict({
        'monitoring_rmse_change_pct': drift_metrics['rmse_change_pct'],
        'monitoring_mae_change_pct': drift_metrics['mae_change_pct'],
        'monitoring_r2_change_pct': drift_metrics['r2_change_pct'],
        'monitoring_alert_count': len(alerts),
        'monitoring_drifted_features': len(drifted_features),
        'monitoring_quality_issues': len(quality_issues)
    })
    
    # Log artifacts
    MLflowLogger.log_dataframe_as_artifact(feature_drift_df, "feature_drift.csv")
    
    print(f"\n✅ Monitoring metrics logged to MLflow")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 13. Monitoring Summary


In [0]:

print("="*60)
print("MONITORING SUMMARY")
print("="*60)

print(f"\n📊 Model: {model_name}")
print(f"📅 Report Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n📈 Performance Status:")
print(f"  • RMSE Change:  {drift_metrics['rmse_change_pct']:+.2f}%")
print(f"  • MAE Change:   {drift_metrics['mae_change_pct']:+.2f}%")
print(f"  • R² Change:    {drift_metrics['r2_change_pct']:+.2f}%")

print(f"\n🔍 Drift Detection:")
print(f"  • Total Features:    {len(feature_drift_df)}")
print(f"  • Drifted Features:  {len(drifted_features)}")
print(f"  • Drift Rate:        {(len(drifted_features)/len(feature_drift_df)*100):.1f}%")

print(f"\n🚨 Alerts:")
if alerts:
    print(f"  • Alert Count: {len(alerts)}")
    for alert in alerts:
        print(f"    - {alert}")
else:
    print(f"  • No alerts")

print(f"\n📋 Data Quality:")
if quality_issues:
    print(f"  • Issue Count: {len(quality_issues)}")
    for issue in quality_issues:
        print(f"    - {issue}")
else:
    print(f"  • No issues detected")

print(f"\n💡 Recommendation:")
print(f"  • {monitoring_report['recommendation']}")

print(f"\n📁 Generated Files:")
print(f"  • monitoring_report_*.json")
print(f"  • feature_drift_*.csv")

print(f"\n✅ Monitoring complete!")
print("="*60)